In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
# import torch

In [2]:
subject_common = pd.read_csv('data/subject_common_info.csv')

In [3]:
subject_common['subject_type'] = "common"
subject_common['subject_grade'] = 1

In [4]:
select_common = ["화법과 작문","독서","언어와 매체","문학","수학Ⅰ","수학Ⅱ","미적분","확률과 통계","영어회화","영어Ⅰ","영어독해와 작문","영어Ⅱ","한국지리","세계지리","세계사","동아시아사","경제","정치와 법","사회·문화","생활과 윤리","윤리와 사상","물리학Ⅰ","화학Ⅰ","생명과학Ⅰ","지구과학Ⅰ","체육","운동과 건강","음악","미술","연극","기술·가정","정보","독일어Ⅰ","한문Ⅰ","철학","논리학","심리학","교육학","종교학","진로와 직업","보건","환경","실용 경제","논술"]

#check text is in select_common
def check_common(text):
    return any(i in text for i in select_common)


In [5]:
#선택교과
subject_names = []
subject_desc = []
subject_type = []
subject_grade = []

for i in range(1,100):
    page = requests.get(f"https://www.cbe.go.kr/dept/19/sub.php?menukey=4496&mod=list1&code1=1&code2={i}")
    soup = bs(page.text, "html.parser")
    soup = soup.find("div",id="contentsType")
    try:
        subject_name = soup.find("h3").text
        subject_names.append(subject_name)
    except Exception:
        continue
    elements = soup.find_all('p')
    subject_desc.append(elements[0].text) # 과목설명
    if check_common(subject_name):
        subject_type.append("select_common")
        subject_grade.append(2)
    else:
        subject_type.append("select_career")
        subject_grade.append(3)
subject_select = pd.DataFrame({"name":subject_names,"description":subject_desc, "subject_type":subject_type, "subject_grade":subject_grade})


In [6]:
subject_select

,name,description,subject_type,subject_grade
0,화법과 작문,초·중·고 공통과목 '국어'의 듣기·말하기 영역과 쓰기 영역을 심화·확장한 과목이다...,select_common,2
1,독서,2009 개정교육과정의 보통 교과 일반 과목이었던 '독서'에서 '독서'부분을 분리한...,select_common,2
2,언어와 매체,2009 개정교육과정의 보통 교과 일반 과목이었던 '독서'에서 '문법'부분을 분리하...,select_common,2
3,문학,"초·중·고 공통 과목 '국어'의 문학 영역을 심화·확장한 과목으로, 다양한 문학 경...",select_common,2
4,실용 국어,초·중·고 공통 과목 '국어'에서 학습한 결과를 바탕으로 하여 주로 취업을 목적으로...,select_career,3
...,...,...,...,...
74,진로와 직업,진로와 직업은 학생이 고등학교 졸업 전 선택해야 할 진로 진학의 문제를 앞서 생각하...,select_common,2
75,보건,"보건은 건강의 가치를 이해하고, 올바른 건강 지식과 자원에 체계적으로 접근하며, 일...",select_common,2
76,환경,"인류가 경험하고 있는 지속불가능성의 확산과 환경위기에 대한 문제의식을 바탕으로, 학...",select_common,2
77,실용 경제,실용 경제는 정보화·세계화의 흐름과 함께 급변하는 현대 경제 환경 속에서 미래의 주...,select_common,2


In [7]:
def crawler_prof(menu_key, code1, start, end):
    for i in range(start,end):
        page = requests.get(f"https://www.cbe.go.kr/dept/19/sub.php?menukey={menu_key}&mod=list2&code1={code1}&code2={i}")
        soup = bs(page.text, "html.parser")
        soup = soup.find("div",id="contentsType")
        try:
            subject_name = soup.find("h3").text
            subject_names.append(subject_name)
        except Exception:
            print(i)
            continue
        elements = soup.find_all('p')
        # print(elements)
        try:
            subject_desc.append(elements[0].text) # 과목설명
        except:
            elements = soup.find_all('li')
            desc = elements[0].text[3:]+elements[1].text[3:]+elements[2].text[3:]
            subject_desc.append(desc) # 과목설명(li 가나다)


In [8]:
#전문교과
subject_names = []
subject_desc = []
subject_grade = []

crawler_prof(4538, 97, 98, 114)
crawler_prof(4538, 114, 115, 129)
crawler_prof(4538, 129, 130, 178)
crawler_prof(4538, 179, 180, 235)
crawler_prof(4538, 236, 237, 248)

subject_type = ["professional_1" for _ in range(len(subject_names))]
subject_grade = [3 for _ in range(len(subject_names))]
subject_professional = pd.DataFrame({"name":subject_names,"description":subject_desc, "subject_type":subject_type,"subject_grade":subject_grade})


In [9]:
subjects = pd.concat([subject_common, subject_select, subject_professional],axis=0)

In [10]:
subjects

,name,description,subject_type,subject_grade
0,국어,국어는 대한민국의 공용어로서 사고와 의사소통의 도구이자 문화 창조와 전승의 기반이다...,common,1
1,통합과학,"‘통합과학’은 자연 현상을 통합적으로 이해하고, 이를 기반으로 자연 현상과 인간의 ...",common,1
2,과학탐구실험,고등학교 ‘과학탐구실험’은 9학년까지의 ‘과학’을 학습한 학생들을 대상으로 하여 과...,common,1
3,한국사,‘한국사’는 우리 역사가 형성·발전되어 온 과정을 이해하여 역사적으로 사고하고 현대...,common,1
4,수학,"수학과는 수학의 개념, 원리, 법칙을 이해하고 기능을 습득하여 주변의 여러 가지 현...",common,1
...,...,...,...,...
139,세계문제와 미래사회,인류가 직면하고 있는 전 지구적 성격의 문제들에 대한 종합적 이해와 미래 사회에 대...,professional_1,3
140,국제관계와 국제기구,국제 관계와 국제기구 과목은 국제 관계의 구조와 질서의 역동적 관계에 대한 이해를 ...,professional_1,3
141,현대 세계의 변화,오늘날 국제 사회에서 발현되는 다양한 현상을 이해하기 위해서는 그러한 현상이 배태된...,professional_1,3
142,사회 탐구 방법,사회 탐구 방법은 인간과 사회 현상에 대한 과학적 탐구 방법의 의미와 특징을 이해하...,professional_1,3


In [11]:
subjects.to_csv('data/subject_info.csv',header=True)